<a href="https://colab.research.google.com/github/dnth/zenml-optuna/blob/master/pytorch_hpo_fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install zenml[server] optuna optuna-dashboard plotly

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
!rm -rf .zen
!zenml init

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim


from zenml.pipelines import pipeline
from zenml.steps import step, Output, BaseParameters

import optuna
from optuna.trial import TrialState
from optuna.storages import RetryFailedTrialCallback

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
@pipeline
def pytorch_hpo_pipeline(
    load_data,
    run_hpo,
    train_test,
):
    """A `pipeline` to load data, load model, and train/evaluate the model."""
    train_dataloader, test_dataloader = load_data()
    best_hparams = run_hpo(train_dataloader, test_dataloader)
    train_test(best_hparams, train_dataloader, test_dataloader)


In [3]:
@step
def load_data() -> Output(
    train_dataloader=DataLoader, test_dataloader=DataLoader
):
    """A `step` to load the Fashion MNIST dataset as a tuple of torch Datasets."""

    batch_size = 64

    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader

In [4]:
def define_hpo_model(trial, hpo_params):
    CLASSES = 10
    n_layers = trial.suggest_int("n_layers", hpo_params.n_layers_low, hpo_params.n_layers_high)
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", hpo_params.hidden_unit_low, hpo_params.hidden_unit_high)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f"dropout_l{i}", hpo_params.dropout_p_low, hpo_params.dropout_p_high)
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def objective(trial, train_dataloader, test_dataloader, hpo_params):

    # Generate model
    model = define_hpo_model(trial, hpo_params)
    
    # Train model
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer_name = trial.suggest_categorical("optimizer_name", hpo_params.optimizer_choices)
    lr = trial.suggest_float("lr", hpo_params.lr_low, hpo_params.lr_high, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_accuracy


class HPOParams(BaseParameters):
    """Optuna Study Params"""
    n_trials: int
    timeout: int
    direction: str
    study_name: str
    sqlite_db_name: str

    optimizer_choices: list
    lr_low: float
    lr_high: float

    n_layers_low:int
    n_layers_high:int

    hidden_unit_low:int
    hidden_unit_high:int

    dropout_p_low:float
    dropout_p_high:float

@step
def run_hpo(
    hpo_params: HPOParams,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> dict:
    
    storage = optuna.storages.RDBStorage(
        hpo_params.sqlite_db_name,
        heartbeat_interval=1,
        failed_trial_callback=RetryFailedTrialCallback(),
    )

    study = optuna.create_study(study_name=hpo_params.study_name, direction=hpo_params.direction, storage=storage)
    study.optimize(lambda trial: objective(trial, train_dataloader, test_dataloader, hpo_params=hpo_params), 
                   n_trials=hpo_params.n_trials, timeout=hpo_params.timeout)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best = trial.params

    print("Found best hyperparam dict from optimization work!")
    print(best)
    
    return best

In [5]:
def define_model(hparam:dict):
    CLASSES = 10
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    print(hparam)
    
    for i in range(hparam['n_layers']):
        out_features = hparam[f"n_units_l{i}"]
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = hparam[f"dropout_l{i}"]
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    """A function to train a model for one epoch."""
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    """A function to test a model on the validation / test dataset."""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_accuracy

In [7]:
@step
def train_test(
    best_hparams: dict,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> Output(trained_model=nn.Module, test_acc=float):
    """A `step` to train and evaluate a torch model on given dataloaders."""
    
    epochs = 5
    
    model = define_model(best_hparams)
    
    print("Constructing optimized model:")
    print(model)

    print("Using optimized hyperparams")
    print(best_hparams)

    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    
    lr = best_hparams['lr']
    optimizer_name = best_hparams['optimizer_name']
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    test_acc = 0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test_acc = test(test_dataloader, model, loss_fn)
    print("Done!")

    return model, test_acc

In [24]:
pytorch_hpo_pipeline(
    load_data=load_data(),
    run_hpo=run_hpo(hpo_params=HPOParams(n_trials=30, timeout=300, direction='maximize', 
                                         study_name="zenml-optuna-hpo", 
                                         sqlite_db_name="sqlite:///zenml-optuna.db", 
                                         optimizer_choices=["Adam", "SGD", "RMSprop"],
                                         lr_low=1e-5,
                                         lr_high=1e-1,
                                         n_layers_low=1,
                                         n_layers_high=5,
                                         hidden_unit_low=8,
                                         hidden_unit_high=128,
                                         dropout_p_low=0.1,
                                         dropout_p_high=0.5
                                        ),
                    ),
    train_test=train_test(),
).run(unlisted=True)

Running unlisted pipeline on stack default (caching enabled)
Step load_data has started.
Using cached version of load_data.
Step load_data has finished in 0.016s.
Step run_hpo has started.


/tmp/ipykernel_69972/1732787293.py:97: ExperimentalWarning:

RetryFailedTrialCallback is experimental (supported from v2.8.0). The interface can change in the future.

[I 2022-11-29 14:00:51,583] A new study created in RDB with name: zenml-optuna-hpo
/home/dnth/anaconda3/envs/zenml-hpo/lib/python3.8/site-packages/optuna/study/_optimize.py:185: ExperimentalWarning:

fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.

[I 2022-11-29 14:00:56,856] Trial 0 finished with value: 60.019999999999996 and parameters: {'n_layers': 1, 'n_units_l0': 46, 'dropout_l0': 0.28570637838799695, 'optimizer_name': 'SGD', 'lr': 0.0016028145179606984}. Best is trial 0 with value: 60.019999999999996.


Test Error: 
 Accuracy: 60.0%, Avg loss: 1.736425 



[I 2022-11-29 14:01:01,885] Trial 1 finished with value: 10.0 and parameters: {'n_layers': 5, 'n_units_l0': 115, 'dropout_l0': 0.1520042393228633, 'n_units_l1': 8, 'dropout_l1': 0.34131471853144335, 'n_units_l2': 53, 'dropout_l2': 0.12444949736186661, 'n_units_l3': 113, 'dropout_l3': 0.25476739019387085, 'n_units_l4': 65, 'dropout_l4': 0.1602869096628317, 'optimizer_name': 'SGD', 'lr': 0.0006696317373710126}. Best is trial 0 with value: 60.019999999999996.


Test Error: 
 Accuracy: 10.0%, Avg loss: 2.306660 



...

Test Error: 
 Accuracy: 82.9%, Avg loss: 0.459815 



[I 2022-11-29 14:03:18,107] Trial 28 finished with value: 84.15 and parameters: {'n_layers': 1, 'n_units_l0': 25, 'dropout_l0': 0.13406863193165305, 'optimizer_name': 'Adam', 'lr': 0.0026586602311428537}. Best is trial 28 with value: 84.15.


Test Error: 
 Accuracy: 84.2%, Avg loss: 0.454946 



[I 2022-11-29 14:03:22,870] Trial 29 finished with value: 82.44 and parameters: {'n_layers': 1, 'n_units_l0': 23, 'dropout_l0': 0.13425122150241958, 'optimizer_name': 'Adam', 'lr': 0.0012157041764616727}. Best is trial 28 with value: 84.15.


Test Error: 
 Accuracy: 82.4%, Avg loss: 0.494390 

Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  0
  Number of complete trials:  30
Best trial:
  Value:  84.15
  Params: 
    dropout_l0: 0.13406863193165305
    lr: 0.0026586602311428537
    n_layers: 1
    n_units_l0: 25
    optimizer_name: Adam
Found best hyperparam dict from optimization work!
{'dropout_l0': 0.13406863193165305, 'lr': 0.0026586602311428537, 'n_layers': 1, 'n_units_l0': 25, 'optimizer_name': 'Adam'}
Step run_hpo has finished in 2m31s.
Step train_test has started.
{'dropout_l0': 0.13406863193165305, 'lr': 0.0026586602311428537, 'n_layers': 1, 'n_units_l0': 25, 'optimizer_name': 'Adam'}
Constructing optimized model:
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=25, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.13406863193165305, inplace=False)
  (4): Linear(in_features=25, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)
Using optim

In [25]:
import optuna
from optuna.storages import RetryFailedTrialCallback

storage = optuna.storages.RDBStorage(
        "sqlite:///zenml-optuna.db",
        heartbeat_interval=1,
        failed_trial_callback=RetryFailedTrialCallback(),
    )

study = optuna.load_study(study_name='zenml-optuna-hpo',storage=storage)

/tmp/ipykernel_69972/2206632645.py:7: ExperimentalWarning:

RetryFailedTrialCallback is experimental (supported from v2.8.0). The interface can change in the future.



In [26]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [27]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study)

In [28]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study)

In [29]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study)

In [30]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study)

In [31]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study)